##### **Disclaimer: We use some advanced packages here without detailed explanation. You can use these, but we do not provide any support.**

In [1]:
# To install them, you can uncomment the following lines:
# (%pip will call pip from the currently active python environment)
# %pip install scikit-learn

# Note: Some of these packages are still not compatible with Python 3.12 yet
# %pip install sweetviz
# %pip install ydata_profiling
# %pip install shap

## CRISP-DM

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn as sns

# Note: The following do not work with Python 3.12
#import shap
#from ydata_profiling import ProfileReport
#import sweetviz as sv

#### Reproducibility 

A best practice in data analytics projects is to work with *seeds* to ensure the reproducability of results. 
This is especially important in the Analytics Cup, since the rules require you to write a self-contained
script that produces reproducable results. 

To achieve this, we can set seeds for all used random number generators.

In [3]:
seed = 55

### Phase 1: Business Understanding

Serves to assess use cases, feasibility, requirements, and
risks of the endeavored data driven project.

Startup that suggests new recipes to users\
But we have been having many cancelations of subscriptions\
Problem was that the users found that the recipes suggested (even though they had high quality) did not match the customer's diet and needs\
Now we have a system of likes and dislikes for the recipes and a new user interface, where the users can enter information about what they want

### Phase 2: Data Understanding

Assess the data quality and content.

In [4]:
# load the data
diet = pd.read_csv("diet.csv")
recipes = pd.read_csv("recipes.csv")
requests = pd.read_csv("requests.csv")
reviews = pd.read_csv("reviews.csv")

C:\Users\sofia\AppData\Local\Temp\ipykernel_18692\1456728697.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv("reviews.csv")


have a look at the data and its attributes \
check if columns are properly named \
general overview over data, check for missing values, etc.

#### Diet pre-processing

In [5]:
diet["Diet"] = diet["Diet"].astype('category')

#### Recipes pre-processing

In [6]:
# Change types of column
def refactorIngredients(ingredients):
    if ingredients == "character(0)":
        return []
    ingredients = ingredients.replace("\\", '').replace("\"", '').replace('c(','').replace(')', '')
    ingredients = ingredients.split(",")
    return ingredients

recipes["RecipeIngredientQuantities"] = recipes["RecipeIngredientQuantities"].apply(lambda x: refactorIngredients(x))
recipes["RecipeIngredientParts"] = recipes["RecipeIngredientParts"].apply(lambda x: refactorIngredients(x))

recipes.head()

,RecipeId,Name,CookTime,PrepTime,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield
0,73440,Bow Ties With Broccoli Pesto,0,1800,Other,"[6, 2, 1 1/2, 1/4, 1/2, 4, 1 1/2, 1 1/2...","[hazelnuts, broccoli florets, fresh parsley ...",241.3,10.1,1.2,0.0,13.1,31.8,2.3,1.4,6.7,9.0,NaN
1,365718,Cashew-chutney Rice,3600,600,Other,"[1, 3/4, 6, 5, 2, 1, 2]","[celery, onion, butter, chicken broth, lon...",370.8,17.5,7.2,22.9,553.3,44.3,1.6,2.2,9.4,8.0,NaN
2,141757,Copycat Taco Bell Nacho Fries BellGrande,3600,2700,Other,"[3, 1/2, 1, 1, 3, 2, 1, 2 1/2, 2, 1, ...","[Copycat Taco Bell Seasoned Beef, yellow onio...",377.6,20.9,10.5,45.7,1501.8,36.6,3.8,6.1,12.9,8.0,NaN
3,280351,Slow Cooker Jalapeno Cheddar Cheese Soup,18000,1800,Other,"[2, 1, 2, 2, 1, 1, 1/8, 1/4, 1, 4, 3...","[unsalted butter, yellow onion, carrots, ga...",282.8,16.5,10.3,50.5,630.2,22.8,2.3,2.7,11.7,6.0,NaN
4,180505,Cool & Crisp Citrus Chiffon Pie,3600,1800,Other,"[1, 1/4, 1/2, 1/2, 1, 1/2, 4, 4, 1/2, ...","[unflavored gelatin, water, sugar, lemon, ...",257.5,8.6,2.4,110.7,160.9,39.8,0.4,30.2,6.3,6.0,NaN


In [7]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75604 entries, 0 to 75603
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    75604 non-null  int64  
 1   Name                        75604 non-null  object 
 2   CookTime                    75604 non-null  int64  
 3   PrepTime                    75604 non-null  int64  
 4   RecipeCategory              75604 non-null  object 
 5   RecipeIngredientQuantities  75604 non-null  object 
 6   RecipeIngredientParts       75604 non-null  object 
 7   Calories                    75604 non-null  float64
 8   FatContent                  75604 non-null  float64
 9   SaturatedFatContent         75604 non-null  float64
 10  CholesterolContent          75604 non-null  float64
 11  SodiumContent               75604 non-null  float64
 12  CarbohydrateContent         75604 non-null  float64
 13  FiberContent                756

In [8]:
# Determines if recipe is veggie, vegan or omnivore
def categorizeRecipe(ingredients):
    meat_derivates = ["pork", "beef", "meat", "fish", "tuna", "chicken", "squid", "schrimp", "trout", "mussels", 
                      "fillet", "lamb", "scallops", "sardine", "salmon", "lobster", "steak", "bacon", "ham", "oyster"]
    animal_derivates = ["milk", "egg", "honey", "gelatin", "butter", "mayonnaise", "cheese", "margarine", 
                    " heavy", "yogurt", "pudding", "shortening", "ice cream", "chocolate", "alfredo", "Miracle Whip", "half-and-half"]
    vegan_exclusions = ["substitute", "peanut", "apple", "vegan", "soymilk"]
    vegan = True
    for ingredient in ingredients:
        if any(word in ingredient.lower() for word in meat_derivates):
            return "Omnivore"
        if ingredient in vegan_exclusions:
            continue
        if any(word in ingredient.lower() for word in animal_derivates):
            vegan = False
    if vegan: 
        return "Vegan"
    else: 
        return "Vegetarian"

recipes["RecipeDiet"] = recipes["RecipeIngredientParts"].apply(lambda x: categorizeRecipe(x))
recipes['RecipeDiet'] = recipes['RecipeDiet'].astype('category')

# Create another table "recipe extra info" columns category, ingredient quatities, parts
selected_columns = ['RecipeCategory', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeServings', 'RecipeYield']
recipe_extra_info = recipes[selected_columns]
recipes = recipes.drop(columns=selected_columns)

recipes

recipe_extra_info.head()


,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,RecipeServings,RecipeYield
0,Other,"[6, 2, 1 1/2, 1/4, 1/2, 4, 1 1/2, 1 1/2...","[hazelnuts, broccoli florets, fresh parsley ...",9.0,NaN
1,Other,"[1, 3/4, 6, 5, 2, 1, 2]","[celery, onion, butter, chicken broth, lon...",8.0,NaN
2,Other,"[3, 1/2, 1, 1, 3, 2, 1, 2 1/2, 2, 1, ...","[Copycat Taco Bell Seasoned Beef, yellow onio...",8.0,NaN
3,Other,"[2, 1, 2, 2, 1, 1, 1/8, 1/4, 1, 4, 3...","[unsalted butter, yellow onion, carrots, ga...",6.0,NaN
4,Other,"[1, 1/4, 1/2, 1/2, 1, 1/2, 4, 4, 1/2, ...","[unflavored gelatin, water, sugar, lemon, ...",6.0,NaN


#### Requests pre-processing

In [9]:
requests.head()

,AuthorId,RecipeId,Time,HighCalories,HighProtein,LowFat,LowSugar,HighFiber
0,2001012259B,73440,1799.950949,0.0,Indifferent,0,0,0
1,437641B,365718,4201.820980,0.0,Yes,0,Indifferent,1
2,1803340263D,141757,6299.861496,0.0,Indifferent,1,Indifferent,0
3,854048B,280351,19801.365796,0.0,Yes,1,0,1
4,2277685E,180505,5400.093457,0.0,Indifferent,0,0,0


In [10]:
requests.info()
# no missing values: GOOD!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140195 entries, 0 to 140194
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   AuthorId      140195 non-null  object 
 1   RecipeId      140195 non-null  int64  
 2   Time          140195 non-null  float64
 3   HighCalories  140195 non-null  float64
 4   HighProtein   140195 non-null  object 
 5   LowFat        140195 non-null  int64  
 6   LowSugar      140195 non-null  object 
 7   HighFiber     140195 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 8.6+ MB


In [11]:
# renaming the columns
requests = requests.rename(columns={"HighCalories": "Calories", "HighProtein":"Protein", "LowFat": "Fat", "LowSugar": "Sugar", "HighFiber":"Fiber"})

In [12]:
# standardizing column Calorie to the same format
requests["Calories"] = requests["Calories"].astype("int")

# standardizing column Protein Yes->1
requests["Protein"] = requests["Protein"].replace("Yes","1")

# changing 0 -> 1 in column Sugar 
requests["Sugar"] = requests["Sugar"].replace("0","1")

# changing 0 -> 1 and 1 -> 0  column Fat
#requests["Fat"] = requests["Fat"].replace({1 : 0, 0 : 1})
requests["Fat"] = 1 - requests["Fat"]

# transforming macronutrients columns -> categories 
#requests[["Calories", "Protein", "Fiber", "Sugar","]] = requests[["Calories", "Protein", "Fiber", "Sugar", "Fat"]].astype("category")

requests


,AuthorId,RecipeId,Time,Calories,Protein,Fat,Sugar,Fiber
0,2001012259B,73440,1799.950949,0,Indifferent,1,1,0
1,437641B,365718,4201.820980,0,1,1,Indifferent,1
2,1803340263D,141757,6299.861496,0,Indifferent,0,Indifferent,0
3,854048B,280351,19801.365796,0,1,0,1,1
4,2277685E,180505,5400.093457,0,Indifferent,1,1,0
...,...,...,...,...,...,...,...,...
140190,163793B,78171,1560.649725,0,Indifferent,1,1,1
140191,33888B,333262,1502.011466,1,Indifferent,0,1,0
140192,401942C,49200,5999.274269,0,Indifferent,1,1,1
140193,346866B,214815,899.523513,0,1,0,Indifferent,1


#### Reviews pre-processing

In [13]:
reviews = reviews.drop(columns = ["Rating"])

### Phase 3: Data Preparation

The goal is assure data quality: includes removing wrong/corrupt 
data entries and making sure the entries are standardized, e.g. enforcing certain encodings. 
Then transforms the data in order to make it suitable for the modelling step. This includes scaling, dimensionality
reduction, data augmentation, outlier removal, etc.\
 \
In practise, this will rarely be the case. On average, this step takes up to **80%** of 
the time of the whole project.

In [14]:
#To do: transform categorical feature into categorical variables (exemplo df["variety"] = df["variety"].astype("category"))
# fill/remove/change missing/corrupt values

# To do: ver se precisamos standardize alguma feature (exemplo na celula seguinte com o StandardScaler), se precisamos imputar valores em registros com valores nulos, 
# se precisamos lidar com outliers, se precisamos usar alguma estretégia de redução de dimensionalidade (tipo PCA na próxima celula)...

Join das 4 tabelas
- há users na tabela "diet" que nao estao na tabela "reviews" -- Ok!
- match perfeito de recipeid and authorid entre requests e reviews -- Otimo!
- todas as receitas de "recipes" estao sendo mostradas para pelo menos um usuario -- Ok!

In [15]:
# tabelas: diet, requests, reviews, recipes
dietrequestsmerged = diet.merge(requests, on = ["AuthorId"])
dietrequestsreviewsmerged = dietrequestsmerged.merge(reviews, on = ["AuthorId", "RecipeId"])
dietrequestsreviewsmerged = dietrequestsreviewsmerged.rename(columns={"Calories" : "Requested_Calories"})
mergedtables = dietrequestsreviewsmerged.merge(recipes, on = ["RecipeId"])
mergedtables = mergedtables.rename(columns={"Calories" : "Recipe_Calories"})

mergedtables["Total_time_recipe"] = mergedtables["CookTime"] + mergedtables["PrepTime"]
mergedtables[["Total_time_recipe", "CookTime", "PrepTime"]]
mergedtables = mergedtables.drop(columns=["PrepTime", "CookTime"])
mergedtables["Time"] = np.where(mergedtables["Time"] < 0, 28_000_000, mergedtables["Time"])
mergedtables["Recipe_Time_Match"] = (mergedtables["Total_time_recipe"] <= (1.2 * mergedtables["Time"]))

mergedtables[(mergedtables["Recipe_Time_Match"] == False)][["Recipe_Time_Match", "Total_time_recipe", "Time"]]
mergedtables = mergedtables.drop(columns=["Time", "Total_time_recipe"])

In [16]:
# categorical_values = ['Diet', 'RecipeDiet', 'Requested_Calories', 'Protein', 'Sugar', 'Fiber']

# fat_labels= [0 ,1]
# bins = [-1, 22.0, np.inf]
# mergedtables["FatCategory"]= pd.cut(mergedtables["FatContent"], bins = bins , labels= fat_labels)
# mergedtables["MatchFat"] = mergedtables["FatCategory"] == mergedtables["Fat"]
# mergedtables = mergedtables.drop(columns=["FatContent", "Fat", "FatCategory"])

# sugar_labels= [0 ,1]
# bins = [-1, 10.0, np.inf]
# mergedtables["SugarCategory"]= pd.cut(mergedtables["SugarContent"], bins = bins , labels= sugar_labels)
# mergedtables["MatchSugar"] = mergedtables["SugarCategory"] == mergedtables["Sugar"]
# mergedtables = mergedtables.drop(columns=["SugarContent", "Sugar", "SugarCategory"])

# protein_labels= [0 ,1]
# bins = [-1, 10.0, np.inf]
# mergedtables["ProteinCategory"]= pd.cut(mergedtables["ProteinContent"], bins = bins , labels= protein_labels)
# mergedtables["MatchProtein"] = mergedtables["ProteinCategory"] == mergedtables["Protein"]
# mergedtables = mergedtables.drop(columns=["ProteinContent", "Protein", "ProteinCategory"])

# fiber_labels= [0 ,1]
# bins = [-1, 5.0, np.inf]
# mergedtables["FiberCategory"]= pd.cut(mergedtables["FiberContent"], bins = bins, labels= fiber_labels)
# mergedtables["MatchFiber"] = mergedtables["FiberCategory"] == mergedtables["Fiber"]
# mergedtables = mergedtables.drop(columns=["FiberContent", "Fiber", "FiberCategory"])


# calories_labels= [0 ,1]
# bins = [-1, 5.0, np.inf]
# mergedtables["CaloriesCategory"]= pd.cut(mergedtables["Recipe_Calories"], bins = bins, labels= calories_labels)
# mergedtables["MatchCalories"] = mergedtables["Requested_Calories"] == mergedtables["Recipe_Calories"]
# mergedtables = mergedtables.drop(columns=["Requested_Calories", "Recipe_Calories", "CaloriesCategory"])'

mergedtables = mergedtables.drop(columns=["SaturatedFatContent", "CholesterolContent", "SodiumContent", "CarbohydrateContent"])


In [17]:
def diet_match(person_diet, recipe_diet):
    if person_diet == "Omnivore":
        return True
    if person_diet == "Vegetarian" and recipe_diet != "Omnivore":
        return True
    if person_diet == "Vegan" and recipe_diet == "Vegan":
        return True
    
    return False

mergedtables["RecipeMatch"] = mergedtables.apply(lambda row: diet_match(row["Diet"], row["RecipeDiet"]), axis= 1)

#mergedtables[["RecipeMatch", "Diet", "RecipeDiet"]]


In [18]:
mergedtables = mergedtables.drop(columns=["Diet", "RecipeDiet"])

Start of feature selection function:

In [19]:
# Drop columns that should not be considered
# Drop Name because is string and Random Forest doesn't accept strings
selected_columns_test = ['AuthorId', 'RecipeId', 'TestSetId', 'Name']
mergedtables = mergedtables.drop(columns= selected_columns_test)

# Drop categorical values and transform them into one column for each of possible categories
# This also removes remaining string values
# ATTENTION: Eu nao sei se essa parte eh necessaria para o Linear Regression. Acredito que sim, mas, se nao, reorganizamos o codigo de repente
categorical_values = ["Requested_Calories", "Protein", "Fat", "Sugar", "Fiber"] # 'Diet', 'RecipeDiet', 

In [20]:
for column in categorical_values:
    new_data = pd.get_dummies(mergedtables[column], prefix=column)
    mergedtables = pd.concat([mergedtables, new_data], axis=1)
            
    mergedtables = mergedtables.drop(columns=column)

In [21]:
mergedtables = mergedtables.drop(columns=["Recipe_Calories", "Sugar_1", "Fat_0"])

In [22]:
#mergedtables = mergedtables.drop(columns=["Protein_1"])

In [23]:
'''
features = mergedtables.columns
results = pd.DataFrame()

iteration = 1
for i in features:
    if i != "Like": # it is not a feature, but the class 
        mergedtablesiteration = mergedtables.drop(columns=[i])

        submissiondataset = mergedtablesiteration[mergedtablesiteration["Like"].isna()] #com Null na coluna Like
        trainandtestdataset = mergedtablesiteration[mergedtablesiteration["Like"].notna()] #sem Null na coluna Like

        # Assuming df is your DataFrame
        df_majority = trainandtestdataset[trainandtestdataset['Like'] == 0]
        df_minority = trainandtestdataset[trainandtestdataset['Like'] == 1]

        length_majority = len(df_majority)
        length_minority = len(df_minority)
        difference = length_majority - length_minority
        final_length = int(difference * 0.7 + length_minority)

        # Downsample the majority class
        df_majority = resample(df_majority, replace=False, n_samples= final_length, random_state=seed)

        # Undersample the minority class
        df_minority = resample(df_minority, replace=True, n_samples= final_length, random_state=seed)

        # Combine the downsampled majority class with the original minority class
        trainandtestdataset = pd.concat([df_minority, df_majority])

        X_features = trainandtestdataset.drop(columns="Like")
        Y_classes = trainandtestdataset["Like"]
        Y_classes = Y_classes.astype('category')


        X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_classes,
                                                            test_size=0.2, 
                                                            shuffle=True,
                                                            random_state=seed) # for reproducibility
        train_df = X_train
        train_df["Y_train"] = Y_train
        #if i != "Recipe_Calories":
        #    train_df = train_df.loc[train_df["Recipe_Calories"] > 0] # OUTLIERS

        to_be_filtered = ["FatContent", "FiberContent", "SugarContent", "ProteinContent"] #"Recipe_Calories", 

        for column in to_be_filtered:
            if column != i:
                good_max_value = train_df[column].mean() + 5 * train_df[column].std() #OUTLIERS
                good_min_value = train_df[column].mean() - 5 * train_df[column].std() #OUTLIERS

                train_df = train_df.loc[(train_df[column] > good_min_value) & (train_df[column] < good_max_value)] #OUTLIERS

        X_train = train_df.drop(columns=["Y_train"])
        Y_train = train_df["Y_train"]

        # TRAIN ################################################################
        random_forest = RandomForestClassifier()
        random_forest.fit(X_train, Y_train)

        # TEST ################################################################
        Y_pred = random_forest.predict(X_test)

        confusion_matrix1 = confusion_matrix(Y_test, Y_pred)

        true_negatives = confusion_matrix1[0][0]
        false_negatives = confusion_matrix1[1][0]
        false_positives = confusion_matrix1[0][1]
        true_positives = confusion_matrix1[1][1]

        sensitivity = true_positives / (true_positives + false_negatives)
        specificity = true_negatives / (true_negatives + false_positives)


        new_row = {"droppedfeature": i, "sensitivity": sensitivity, "specificity": specificity, "mean": (sensitivity + specificity)/2}
        results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)

        iteration = iteration + 1
'''        


'\nfeatures = mergedtables.columns\nresults = pd.DataFrame()\n\niteration = 1\nfor i in features:\n    if i != "Like": # it is not a feature, but the class \n        mergedtablesiteration = mergedtables.drop(columns=[i])\n\n        submissiondataset = mergedtablesiteration[mergedtablesiteration["Like"].isna()] #com Null na coluna Like\n        trainandtestdataset = mergedtablesiteration[mergedtablesiteration["Like"].notna()] #sem Null na coluna Like\n\n        # Assuming df is your DataFrame\n        df_majority = trainandtestdataset[trainandtestdataset[\'Like\'] == 0]\n        df_minority = trainandtestdataset[trainandtestdataset[\'Like\'] == 1]\n\n        length_majority = len(df_majority)\n        length_minority = len(df_minority)\n        difference = length_majority - length_minority\n        final_length = int(difference * 0.7 + length_minority)\n\n        # Downsample the majority class\n        df_majority = resample(df_majority, replace=False, n_samples= final_length, random_

In [39]:
#results.sort_values("mean")

,droppedfeature,sensitivity,specificity,mean
0,Age,0.977271,0.891354,0.934312
10,Fat_1,0.983754,0.906302,0.945028
9,Protein_Indifferent,0.983914,0.910157,0.947035
1,FatContent,0.985914,0.910471,0.948193
4,ProteinContent,0.987195,0.913225,0.950210
6,RecipeMatch,0.987435,0.914326,0.950881
2,FiberContent,0.989676,0.915113,0.952394
3,SugarContent,0.990476,0.916450,0.953463
11,Sugar_Indifferent,0.988635,0.921013,0.954824
13,Fiber_1,0.988876,0.921564,0.955220


In [36]:
#FOURTH ITERATION - GOT WORSE
#results.sort_values("mean")

,droppedfeature,sensitivity,specificity,mean
0,Age,0.976791,0.887971,0.932381
11,Fat_1,0.982953,0.904964,0.943959
1,FatContent,0.985914,0.907560,0.946737
6,RecipeMatch,0.987515,0.911494,0.949504
4,ProteinContent,0.986475,0.913697,0.950086
2,FiberContent,0.990396,0.913146,0.951771
3,SugarContent,0.990236,0.916922,0.953579
12,Sugar_Indifferent,0.988635,0.919125,0.953880
13,Fiber_0,0.988315,0.920541,0.954428
5,Recipe_Time_Match,0.987995,0.921092,0.954544


In [32]:
#THIRD ITERATION
#results.sort_values("mean")

,droppedfeature,sensitivity,specificity,mean
0,Age,0.977591,0.888758,0.933174
1,FatContent,0.986074,0.909842,0.947958
4,ProteinContent,0.987355,0.913067,0.950211
6,RecipeMatch,0.987435,0.912989,0.950212
2,FiberContent,0.990396,0.914011,0.952204
3,SugarContent,0.990396,0.914641,0.952519
13,Sugar_Indifferent,0.989196,0.917394,0.953295
8,Requested_Calories_1,0.987995,0.921800,0.954898
15,Fiber_1,0.988715,0.921249,0.954982
7,Requested_Calories_0,0.988555,0.921643,0.955099


In [29]:
# SECOND ITERATION!!!
#results.sort_values("mean")

,droppedfeature,sensitivity,specificity,mean
0,Age,0.977671,0.887735,0.932703
1,FatContent,0.985514,0.907088,0.946301
4,ProteinContent,0.986795,0.911651,0.949223
6,RecipeMatch,0.988715,0.909842,0.949279
2,FiberContent,0.989996,0.911258,0.950627
3,SugarContent,0.990556,0.914169,0.952363
15,Fiber_0,0.989356,0.918024,0.953690
14,Sugar_Indifferent,0.989116,0.918496,0.953806
9,Protein_1,0.988555,0.919519,0.954037
5,Recipe_Time_Match,0.988796,0.919676,0.954236


In [24]:
#FIRST ITERATION
#results.sort_values("mean")
#new_row

,droppedfeature,sensitivity,specificity,mean
0,Age,0.963906,0.892062,0.927984
7,RecipeMatch,0.978231,0.910629,0.944430
5,ProteinContent,0.978391,0.916765,0.947578
3,FiberContent,0.981993,0.919440,0.950716
16,Fiber_0,0.979352,0.922980,0.951166
8,Requested_Calories_0,0.978151,0.924947,0.951549
11,Protein_Indifferent,0.979272,0.923924,0.951598
2,FatContent,0.983914,0.919283,0.951598
10,Protein_1,0.978792,0.924475,0.951633
9,Requested_Calories_1,0.977831,0.925498,0.951664


#### Sampling

Split our data set into *train* and *test* data set.

In [25]:
# TODO: ver se vamos usar um split para validação, ou usar cross validation

In [25]:
submissiondataset = mergedtables[mergedtables["Like"].isna()] #com Null na coluna Like
trainandtestdataset = mergedtables[mergedtables["Like"].notna()] #sem Null na coluna Like

In [26]:
# Drop columns that should not be considered
# Drop Name because is string and Random Forest doesn't accept strings
selected_columns_test = ['AuthorId', 'RecipeId', 'TestSetId', 'Name']
test_extra_info = trainandtestdataset[selected_columns_test]
submission_extra_info = submissiondataset[selected_columns_test]

submissiondataset = submissiondataset.drop(columns= selected_columns_test)
trainandtestdataset = trainandtestdataset.drop(columns=selected_columns_test)

# Drop categorical values and transform them into one column for each of possible categories
# This also removes remaining string values
# ATTENTION: Eu nao sei se essa parte eh necessaria para o Linear Regression. Acredito que sim, mas, se nao, reorganizamos o codigo de repente
categorical_values = ["Requested_Calories", "Protein", "Fat", "Sugar", "Fiber"] # 'Diet', 'RecipeDiet', 

for column in categorical_values:
    new_data = pd.get_dummies(trainandtestdataset[column], prefix=column)
    trainandtestdataset = pd.concat([trainandtestdataset, new_data], axis=1)

    new_data = pd.get_dummies(submissiondataset[column], prefix=column)
    submissiondataset = pd.concat([submissiondataset, new_data], axis=1)
    
trainandtestdataset = trainandtestdataset.drop(columns=categorical_values)
submissiondataset = submissiondataset.drop(columns=categorical_values)

#submissiondataset.head()


KeyError: "None of [Index(['AuthorId', 'RecipeId', 'TestSetId', 'Name'], dtype='object')] are in the [columns]"

In [29]:
# Separate train and test data and X and Y variables
X_features = trainandtestdataset.drop(columns="Like")
Y_classes = trainandtestdataset["Like"]
Y_classes = Y_classes.astype('category')

#trainandtestdataset.info()

X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_classes,
                                                    test_size=0.2, 
                                                    shuffle=True,
                                                    random_state=seed) # for reproducibility
train_df = X_train
train_df["Y_train"] = Y_train
#train_df = train_df.loc[train_df["Recipe_Calories"] > 0]

to_be_filtered = ["FatContent", "FiberContent", "SugarContent", "ProteinContent"] #"Recipe_Calories", 

for column in to_be_filtered:
    good_max_value = train_df[column].mean() + 5 * train_df[column].std()
    good_min_value = train_df[column].mean() - 5 * train_df[column].std()

    train_df = train_df.loc[(train_df[column] > good_min_value) & (train_df[column] < good_max_value)]

X_train = train_df.drop(columns=["Y_train"])
Y_train = train_df["Y_train"]

- X_train: 77.904 rows × 24 columns
- Y_train: 77.904 rows
- X_test: 19.477 rows × 24 columns
- Y_test: 19.477 rows

### Phase 4: Modeling

In this phase, the model is trained and tuned.

#### Logistic Regression

In [28]:
#trying to adjust feature balance
logistic_regression = LogisticRegression(class_weight='balanced' )

logistic_regression.fit(X_train, Y_train)

Y_pred = logistic_regression.predict(X_test)

c:\Users\sofia\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Random Forest

In [32]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

Threshold Adjustment

##### Random Forest Analysis

Comparison between predictions 

Adjusted 
-accuracy: 0.87
-sensitivity =  0.72
-specificity =  0.89

No adjusted:
-accuracy: 0.90
-sensitivity =  0.44
-specificity =  0.97

In [33]:
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)
confusion_matrix = confusion_matrix(Y_test, Y_pred)
print(confusion_matrix)

true_negatives = confusion_matrix[0][0]
false_negatives = confusion_matrix[1][0]
false_positives = confusion_matrix[0][1]
true_positives = confusion_matrix[1][1]

sensitivity = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)

print("sensitivity = ", sensitivity)
print("specificity = ", specificity)

# Too many False predictions

# Possible ways to improve
# Re add the recipe name in some way - parse the string and see if the title is vegetarian. 
# Group the cook time in discrete chunks?
# Group the other nutritional facts columns of recipe in discrete chunks?
# Group age in chunks ?
# Drop some columns from recipe like sodium 
# Reduce dimensionality. I guess fat, saturated fat and cholesterol are correlated.


Accuracy: 0.8976741798018175
[[16472   463]
 [ 1530  1012]]
sensitivity =  0.3981117230527144
specificity =  0.9726601712429879


In [33]:
tabela = X_test
tabela["Like"] = Y_test
tabela["Pred"] = Y_pred_adjusted
tabela[(tabela["Pred"] == 1) & (tabela["Like"] == False)]

,Age,Recipe_Calories,FatContent,FiberContent,SugarContent,ProteinContent,Recipe_Time_Match,RecipeMatch,Requested_Calories_0,Requested_Calories_1,Protein_1,Protein_Indifferent,Fat_0,Fat_1,Sugar_1,Sugar_Indifferent,Fiber_0,Fiber_1,Like,Pred
105774,56,344.9,7.5,12.6,0.8,19.7,True,True,True,False,False,True,False,True,True,False,True,False,False,1
97988,72,477.3,0.7,2.7,107.0,2.3,True,True,True,False,True,False,True,False,True,False,False,True,False,1
16065,68,838.9,49.4,9.8,23.4,16.1,True,True,True,False,False,True,False,True,True,False,False,True,False,1
73732,67,488.1,26.3,2.0,36.5,7.0,True,True,False,True,True,False,False,True,False,True,True,False,False,1
125704,49,3145.4,88.2,19.0,41.3,96.2,True,True,False,True,False,True,False,True,True,False,True,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20099,65,1077.5,76.2,1.7,59.6,15.8,True,False,False,True,False,True,False,True,True,False,False,True,False,1
64096,18,4283.0,361.8,13.6,96.4,98.4,True,False,False,True,False,True,False,True,False,True,True,False,False,1
126010,55,551.8,16.9,10.2,47.5,35.1,True,True,False,True,True,False,False,True,False,True,False,True,False,1
85751,38,678.8,42.6,6.4,3.6,15.1,True,True,False,True,False,True,False,True,False,True,False,True,False,1


In [34]:
tabela.groupby(["Like", "Pred"]).mean()

C:\Users\sofia\AppData\Local\Temp\ipykernel_2916\3884456061.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tabela.groupby(["Like", "Pred"]).mean()


Age  Recipe_Calories  FatContent  FiberContent  \
Like  Pred                                                         
False 0     44.285686       441.534310   23.456018      3.332652   
      1     58.194069       789.492716   39.265921      5.981530   
True  0     51.652835       498.903181   23.655187      4.485477   
      1     62.271578       738.917152   33.315118      6.035349   

            SugarContent  ProteinContent  Recipe_Time_Match  RecipeMatch  \
Like  Pred                                                                 
False 0        17.855672       17.121168                1.0     0.729768   
      1        43.928772       23.638970                1.0     0.882414   
True  0        21.939004       18.845367                1.0     0.775934   
      1        42.349258       22.654865                1.0     0.946674   

            Requested_Calories_0  Requested_Calories_1  Protein_1  \
Like  Pred                                                          
False 0                 0.586758              0.413242   0.414441   
      1                 0.685224              0.314776   0.313215   
True  0                 0.683264              0.316736   0.369295   
      1                 0.626168              0.373832   0.389225   

            Protein_Indifferent     Fat_0     Fat_1   Sugar_1  \
Like  Pred                                                      
False 0                0.585559  0.303004  0.696996  0.714447   
      1                0.686785  0.253902  0.746098  0.659729   
True  0                0.630705  0.242047  0.757953  0.648686   
      1                0.610775  0.350742  0.649258  0.626718   

            Sugar_Indifferent   Fiber_0   Fiber_1  
Like  Pred                                         
False 0              0.285553  0.594551  0.405449  
      1              0.340271  0.622789  0.377211  
True  0              0.351314  0.604426  0.395574  
      1              0.373282  0.603628  0.396372

#### Submission

In [35]:
X_features_submission = submissiondataset.drop(columns="Like")
X_features_submission.head()

,Age,Recipe_Calories,FatContent,FiberContent,SugarContent,ProteinContent,Recipe_Time_Match,RecipeMatch,Requested_Calories_0,Requested_Calories_1,Protein_1,Protein_Indifferent,Fat_0,Fat_1,Sugar_1,Sugar_Indifferent,Fiber_0,Fiber_1
5,52,395.7,19.2,0.8,4.3,16.3,True,False,False,True,False,True,False,True,True,False,True,False
15,37,104.4,8.2,2.0,4.0,2.1,True,True,False,True,False,True,False,True,True,False,True,False
20,55,239.1,12.1,0.7,24.1,2.4,True,True,False,True,True,False,False,True,True,False,False,True
22,61,239.1,12.1,0.7,24.1,2.4,True,True,True,False,False,True,False,True,False,True,True,False
23,45,239.1,12.1,0.7,24.1,2.4,True,True,True,False,False,True,False,True,True,False,True,False


In [36]:
# submissiion

# Let's assume that our id column is the index of the dataframe

id = submission_extra_info['TestSetId']
Y_pred_submission = random_forest.predict(X_features_submission)

output = pd.DataFrame({'id': id, 'prediction': Y_pred_submission})

#output = output.rename(columns={'TestSetId': 'id'})

#output
output.info()
output['id'] = output["id"].astype('int')
output['prediction'] = output["prediction"].astype('int')
output.to_csv('analzticscuppredictionfile.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 42814 entries, 5 to 140189
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          42814 non-null  float64
 1   prediction  42814 non-null  float64
dtypes: float64(2)
memory usage: 1003.5 KB


In [37]:
output.head()

,id,prediction
5,41190,0
15,18123,0
20,36379,0
22,33658,0
23,24872,0


In [38]:
# Here, you want to find the best classifier. As candidates, consider
#   1. LogisticRegression
#   2. RandomForestClassifier
#   3. other algorithms from sklearn (easy to add)
#   4. custom algorithms (more difficult to implement)
    
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

model_logistic_regression = LogisticRegression(max_iter=30)
model_random_forest = RandomForestClassifier()
model_gradient_boosting = GradientBoostingClassifier()

# train the models
pipeline = Pipeline(steps=[("scaler", transform_scaler), 
                           ("pca", transform_pca),
                           ("model", None)])

parameter_grid_preprocessing = {
  "pca__n_components" : [1, 2, 3, 4],
}

parameter_grid_logistic_regression = {
  "model" : [model_logistic_regression],
  "model__C" : [0.1, 1, 10],  # inverse regularization strength
}

parameter_grid_gradient_boosting = {
  "model" : [model_gradient_boosting],
  "model__n_estimators" : [10, 20, 30]
}

parameter_grid_random_forest = {
  "model" : [model_random_forest],
  "model__n_estimators" : [10, 20, 50],  # number of max trees in the forest
  "model__max_depth" : [2, 3, 4],
}

meta_parameter_grid = [parameter_grid_logistic_regression,
                       parameter_grid_random_forest,
                       parameter_grid_gradient_boosting]

meta_parameter_grid = [{**parameter_grid_preprocessing, **model_grid}
                       for model_grid in meta_parameter_grid]

search = GridSearchCV(pipeline,
                      meta_parameter_grid, 
                      scoring="balanced_accuracy",
                      n_jobs=2, 
                      cv=5,  # number of folds for cross-validation 
                      error_score="raise"
)

# here, the actual training and grid search happens
search.fit(X_train, Y_train.values.ravel())

print("best parameter:", search.best_params_ ,"(CV score=%0.3f)" % search.best_score_)

NameError: name 'transform_scaler' is not defined

### Step 5: Evaluation

Once the appropriate models are chosen, they are evaluated on the test set. For
this, different evaluation metrics can be used. Furthermore, this step is where
the models and their predictions are analyzed resp. different properties, including
feature importance, robustness to outliers, etc.

In [ ]:
# evaluate performance of model on test set
print("Score on test set:", search.score(X_test, Y_test.values.ravel()))

# contingency table
ct = pd.crosstab(search.best_estimator_.predict(X_test), Y_test.values.ravel(),
                 rownames=["pred"], colnames=["true"])
print(ct)

In [ ]:
# (optional, if you're curious) 
# for a detailed look on the performance of the different models
def get_search_score_overview():
  for c,s in zip(search.cv_results_["params"],search.cv_results_["mean_test_score"]):
      print(c, s)

print(get_search_score_overview())

#### Interpretability

##### Disclaimer: This only works if shap is installed.

In addition to models and their predictions, it is often important to understand _why_ a model makes certain predictions. 
There is a lot of literature on how this can be achieved (explainability), but we will only show the use of Shapley values
using the python module "shap", which is a combination of Shapley values and LIME. 
You can find more information on this topic [here](https://christophm.github.io/interpretable-ml-book/shap.html).

In [ ]:
# assume random forest model
model = RandomForestClassifier(n_estimators=10, random_state=seed)
model.fit(X_train, Y_train.values.ravel())

# compute shapley values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_interaction_values = explainer.shap_interaction_values(X_train)

expected_value = explainer.expected_value
print(expected_value)

In [ ]:
# class dependent plots of shapley values for each feature
for i,c in enumerate(df.variety.unique()):
    shap.summary_plot(shap_values[i], X_train, show=False)
    plt.title("Shapley values for "+str(c))
    plt.show()

From the computed SHAP values, we can interpret that the *petal.width* has a positive impact on the output of the model 
if the feature value is moderate. For high aand low values, the impact is negative. The same observation
holds for *petal.length*. Besides, the impact of the *sepal.length* and *sepal.width* features are rather low. By impact on a 
the target, we model the probability that we classify that target. Thus, if *petal.width* is high, it is more likely
that we classify the data point as Versicolor.

### Step 6: Deployment

Now that you have chosen and trained your model, it is time to deploy it to your
clients system. 

In [ ]:
def micro_service_classify_iris(datapoint):
    
  # make sure the provided datapoints adhere to the correct format for model input

  # fetch your trained model
  model = search.best_estimator_

  # make prediction with the model
  prediction = model.predict(datapoint)

  return prediction


In the Analytics Cup, you need to export your prediction in a very specific output format. This is a csv file without an index and two columns, *id* and *prediction*. Note that the values in both columns need to be integer values, and especially in the *prediction* column either 1 or 0.

In [ ]:
# To do: arrumar a celula abaixo com os nossos dataframes